# Test Azure Document Intelligence

In [1]:
#Import libraries
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.storage.blob import BlobServiceClient, BlobClient
from dotenv import load_dotenv
import os

In [2]:
# Get Configuration Settings
load_dotenv()
document_intelligence_endpoint = os.getenv('DOCUMENT_INTELLIGENCE_ENDPOINT')
document_intelligence_key = os.getenv('DOCUMENT_INTELLIGENCE_API_KEY')
model_id = os.getenv('DOCUMENT_INTELLIGENCE_MODEL')
# storage_connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
# container_name = os.getenv('AZURE_STORAGE_CONTAINER_NAME')
# blob_name = os.getenv('AZURE_STORAGE_BLOB_NAME')

In [4]:
#Routes
# path_to_sample_documents = r'C:\Users\julio\Downloads\Muestra boletas\BOLETA MARZO 24.jpg'
path_to_sample_documents = r"C:\Users\julio\Downloads\Muestras de cronogramas\CRONOGRAMA DE PAGOS DE BCP.pdf"

In [5]:
# # Crear cliente de servicio de blobs y cliente de blob específico
# blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
# blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# # Leer el contenido del blob
# with blob_client.download_blob() as download_stream:
#     blob_content = download_stream.readall()

# Crear cliente de inteligencia de documentos
document_intelligence_client = DocumentIntelligenceClient(endpoint=document_intelligence_endpoint, 
                                                          credential=AzureKeyCredential(document_intelligence_key)
                                                          )

# Leer el contenido del archivo
with open(path_to_sample_documents, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        model_id= model_id, 
        analyze_request= f, 
        content_type= "application/octet-stream", 
        features= ["keyValuePairs"]
    )

# # Enviar el contenido del blob al cliente de inteligencia de documentos
# poller = document_intelligence_client.begin_analyze_document(
#     model_id= model_id, analyze_request= blob_content, content_type= "application/octet-stream"
# )
result: AnalyzeResult = poller.result()


In [6]:
for table_idx, table in enumerate(result.tables):
    print(type(table))

<class 'azure.ai.documentintelligence.models._models.DocumentTable'>
<class 'azure.ai.documentintelligence.models._models.DocumentTable'>
<class 'azure.ai.documentintelligence.models._models.DocumentTable'>
<class 'azure.ai.documentintelligence.models._models.DocumentTable'>
<class 'azure.ai.documentintelligence.models._models.DocumentTable'>
<class 'azure.ai.documentintelligence.models._models.DocumentTable'>


In [ ]:
paragraph_dict = {}
if result.paragraphs:
    consolidated_content = "\n".join(item['content'] for item in result.paragraphs)

In [ ]:
# Extract the content and bounding regions
texts = [(item['content'], item['boundingRegions'][0]['polygon']) for item in result.paragraphs]

# Function to calculate the center of a bounding box
def get_center(polygon):
    x_coords = [polygon[i] for i in range(0, len(polygon), 2)]
    y_coords = [polygon[i+1] for i in range(0, len(polygon), 2)]
    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)
    return (center_x, center_y)

# Get centers for each text
texts_with_centers = [(text, get_center(polygon)) for text, polygon in texts]

# Sort texts by their y-coordinate (and then x-coordinate to handle lines)
texts_with_centers.sort(key=lambda item: (item[1][1], item[1][0]))

# Combine the texts into a single string
consolidated_text = '/n'.join([text for text, center in texts_with_centers])

print(consolidated_text)



In [ ]:
print(consolidated_content)

In [ ]:
# Extract the content and bounding regions
texts = [(item['content'], item['boundingRegions'][0]['polygon']) for item in result.paragraphs]

# Function to calculate the center of a bounding box
def get_center(polygon):
    x_coords = [polygon[i] for i in range(0, len(polygon), 2)]
    y_coords = [polygon[i+1] for i in range(0, len(polygon), 2)]
    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)
    return (center_x, center_y)

# Get centers for each text
texts_with_centers = [(text, get_center(polygon)) for text, polygon in texts]

# Sort texts by their y-coordinate (and then x-coordinate to handle lines)
texts_with_centers.sort(key=lambda item: (item[1][1], item[1][0]))

# Define the proximity threshold
proximity_threshold_y = 25  # Adjust this value based on your data
proximity_threshold_x = 125  # Adjust this value based on your data

# Group texts that are close to each other
grouped_texts = []
current_group = []

for i in range(len(texts_with_centers)):
    text, center = texts_with_centers[i]
    
    if not current_group:
        current_group.append((text, center))
    else:
        last_text, last_center = current_group[-1]
        
        if abs(center[1] - last_center[1]) < proximity_threshold_y and abs(center[0] - last_center[0]) < proximity_threshold_x:
            current_group.append((text, center))
        else:
            grouped_texts.append(current_group)
            current_group = [(text, center)]

if current_group:
    grouped_texts.append(current_group)

# Consolidate the texts
consolidated_text = '\n'.join([' '.join([text for text, center in group]) for group in grouped_texts])

print(consolidated_text)


In [ ]:
import re
def clean_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.replace(':', '')
    text = text.strip()
    return text

In [ ]:
kv_dict = {}
if result.key_value_pairs:
    for kv_pair in result.key_value_pairs:
        if kv_pair.key and kv_pair.value:
            key = clean_text(kv_pair.key.content)
            kv_dict[key] = kv_pair.value.content

In [ ]:
import json
a = json.dumps(kv_dict)
print(json.dumps(kv_dict))

In [ ]:
table_dict = {}
for table_idx, table in enumerate(result.tables):
    aux_list = []
    for cell in table.cells:
        aux_dict = {}
        aux_dict['row'] = cell.row_index
        aux_dict['column'] = cell.column_index
        aux_dict['content'] = cell.content.encode("utf-8").decode("utf-8")
        if aux_dict['content'] != "":
            aux_list.append(aux_dict)
    table_dict[table_idx] = aux_list

In [ ]:
import json
a = json.dumps(table_dict)
print(a)

In [ ]:
table_dict[3]

In [ ]:
b = ""
a = b.encode("utf-8").decode("utf-8")

In [ ]:
import json

# Cargar el archivo JSON
with open(r'C:\Users\julio\Downloads\CRONOGRAMA DE PAGOS DE BCP.pdf.json', 'r') as file:
    data = json.load(file)

# Inicializar listas para almacenar las regiones de tablas y los párrafos no en tablas
table_regions = []
paragraphs = []

# Recopilar regiones de las tablas
for table in data['analyzeResult'].get('tables', []):
    for cell in table.get('cells', []):
        for region in cell.get('boundingRegions', []):
            table_regions.append(region)

# Filtrar párrafos que no están dentro de las tablas
for paragraph in data['analyzeResult'].get('paragraphs', []):
    in_table = False
    for region in paragraph.get('boundingRegions', []):
        for table_region in table_regions:
            # Comparar regiones (puedes ajustar la lógica de comparación según tus necesidades)
            if (region['pageNumber'] == table_region['pageNumber'] and
                region['polygon'][0] >= table_region['polygon'][0] and
                region['polygon'][1] >= table_region['polygon'][1] and
                region['polygon'][2] <= table_region['polygon'][2] and
                region['polygon'][3] <= table_region['polygon'][3]):
                in_table = True
                break
        if in_table:
            break
    if not in_table:
        paragraphs.append(paragraph['content'])

# Mostrar los párrafos filtrados
for paragraph in paragraphs:
    print(paragraph)


In [ ]:
import json

# Cargar el archivo JSON
with open(r'C:\Users\julio\Downloads\CRONOGRAMA DE PAGOS DE BCP.pdf.json', 'r') as file:
    data = json.load(file)

# Inicializar listas para almacenar las regiones de tablas y los párrafos no en tablas
table_regions = []
paragraphs = []

# Recopilar regiones de las tablas
for table in data['analyzeResult'].get('tables', []):
    for cell in table.get('cells', []):
        for region in cell.get('boundingRegions', []):
            table_regions.append(region)

# Filtrar párrafos que no están dentro de las tablas
for paragraph in data['analyzeResult'].get('paragraphs', []):
    paragraphs.append(paragraph['content'])

# Mostrar los párrafos filtrados
for paragraph in paragraphs:
    print(paragraph)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
import httpx

template = """
tell me 4 jokes about {topic}
"""

functions = [
    {
        "name": "jokes",
        "description": "jokes",
        "parameters": {
            "type": "object",
            "properties": {
                "joke1": {
                    "type": "string", 
                    "description": "joke1"
                },
                "joke2": {
                    "type": "string",
                    "description": "joke2",
                },
                "joke3":{
                    "type": "string",
                    "description": "joke3"
                },
                "joke4":{
                    "type": "string",
                    "description": "joke4"
                },
            },
            "required": ["joke1", "joke2", "joke3", "joke4"],
        },
    }
]

function_name = "jokes"


In [ ]:
ocr_content = """
bears
"""
ocr_content = ocr_content.encode("utf-8").decode("utf-8")

In [ ]:
from langchain.globals import set_debug

set_debug(True)


In [ ]:

prompt = ChatPromptTemplate.from_template(template)
# # Create an HTTP client that does not verify SSL certificates
# client = httpx.Client(verify=False)
model = AzureChatOpenAI(
    openai_api_type='azure',
    openai_api_version="2024-02-15-preview",
    azure_deployment="oaisaltoibk35T16k",
    azure_endpoint="https://saltoeusibkoai01dev.openai.azure.com/",
    openai_api_key="d0a66ddc2ce24a0b96d1ab206233f079",
    # http_client= client,
)
# Build chain
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": function_name}, functions=functions)
    | JsonOutputFunctionsParser()
)
# Invoke Azure OpenAI
result = chain.invoke(ocr_content)
 


In [ ]:
from langchain.globals import set_debug

set_debug(False)
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
model.invoke([message])

In [ ]:
print(result)

In [ ]:
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

class Payment_Information(BaseModel):
    """Joke to tell user."""

    bank: str = Field(description="The Bank that issued the loan")
    monthly_payment: str = Field(description="The monthly payment amount the user pays for their loan.")
    total_loan_amount: str = Field(description="The total amount loaned by the bank.")
    interest_rate: str = Field(description="Find the interest rate at which the loan was issued.")


openai_functions = [convert_pydantic_to_openai_function(Payment_Information)]

In [ ]:
template = """
## Task Goal
The task goal is to identify specific financial details from the extracted OCR text of a payment schedule.

## Task Instructions
You will be given a text extracted with OCR from a payment schedule in Spanish. The text contains plain text, key-value pairs and tables. Please identify and extract the following elements:

1. **Bank**: Identify the bank that issued the loan. Some known banks include: "BCP" or "Banco de crédito del Perú", "BBVA", "Scotiabank", "Interbank", "Banco Pichincha", "Mi Banco", "BanBif", "Banco Falabella", "Banco Ripley", "Banco Santander", "Alfin Banco", "BCI", etc.
2. **Monthly Payment**: Find the monthly payment amount the user pays for their loan. It may appear as "cuota", "monto", "cuota mensual", "próximo pago", etc.
3. **Total Loan Amount**: Identify the total amount loaned by the bank. It may appear as "importe de préstamo", "monto original", "crédito hipotecario", etc.
4. **Interest Rate**: Find the interest rate at which the loan was issued. It may appear as "tasa de costo efectivo", "tasa anual", "TEA", "TCEA", etc.

If you cannot find a specific value, return an empty string ("").

## Task Output:
Provide the extracted information as follows:
- Bank: [Bank Name]
- Monthly Payment: [Monthly Payment Amount]
- Total Loan Amount: [Total Loan Amount]
- Interest Rate: [Interest Rate]

## Task Input:
{ocr_content}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)
# # Create an HTTP client that does not verify SSL certificates
# client = httpx.Client(verify=False)
model = AzureChatOpenAI(
    openai_api_type='azure',
    openai_api_version="2024-02-15-preview",
    azure_deployment="oaisaltoibk35T16k",
    azure_endpoint="https://saltoeusibkoai01dev.openai.azure.com/",
    openai_api_key="d0a66ddc2ce24a0b96d1ab206233f079",
    # http_client= client,
)

In [ ]:
parser = JsonOutputFunctionsParser()
chain = prompt | model.bind(functions=openai_functions) | parser

In [ ]:
ocr_content = """

                PARAGRAPHS:
                BCP> BCP> BCP> BCP> PAGINA 2 PAGINA 2 PAGINA 1 PAGINA 1 AMR261P2-EFCM AMR261P2-EFCM AMR261P2-EFCM AMR261P2-EFCM USCACHI SANTOS EDITH USCACHI SANTOS EDITH USCACHI SANTOS EDITH USCACHI SANTOS EDITH JR. JOSE CARLOS MARIATEGUI MZ. C LT. 6 URB JR. JOSE CARLOS MARIATEGUI MZ. C LT. 6 URB JR. JOSE CARLOS MARIATEGUI MZ. C LT. 6 URB JR. JOSE CARLOS MARIATEGUI MZ. C LT. 6 URB 1687 1687 1687 1687 CUZCO-CUZCO-WANCHAQ CUZCO-CUZCO-WANCHAQ CUZCO-CUZCO-WANCHAQ CUZCO-CUZCO-WANCHAQ 100-285-0000-00000009400201 100-285-0000-00000009400201 100-285-0000-00000009400201 100-285-0000-00000009400201 CREDITO NRO : CREDITO NRO : CREDITO NRO : CREDITO NRO : : : SOLES SOLES SOLES SOLES PRODUCTO PRODUCTO CREDITO NEGOCIOS CREDITO NEGOCIOS PRODUCTO : CREDITO NEGOCIOS IMPORTE DESEMBOLSADO 240,950.00 : PRODUCTO : CREDITO NEGOCIOS IMPORTE DESEMBOLSADO 240,950.00 : 240,950.00 240,950.00 IMPORTE DESEMBOLSADO IMPORTE DESEMBOLSADO : : IMPORTE PRESTAMO IMPORTE PRESTAMO IMPORTE PRESTAMO IMPORTE PRESTAMO 240,950.00 240,950.00 240,950.00 240,950.00 : : : : TASA DE INTERES COMPENSATORIA EFECTIVA ANUAL (1) : TASA DE INTERES COMPENSATORIA EFECTIVA ANUAL (1) : TASA DE INTERES COMPENSATORIA EFECTIVA ANUAL (1) : TASA DE INTERES COMPENSATORIA EFECTIVA ANUAL (1) : 12.70% 12.70% 12.70% 12.70% TASA DE COSTO EFECTIVO ANUAL REMANENTE TASA DE COSTO EFECTIVO ANUAL REMANENTE TASA DE COSTO EFECTIVO ANUAL REMANENTE TASA DE COSTO EFECTIVO ANUAL REMANENTE 14.17% 14.17% 14.17% 14.17% : : 1.08% TASA ANUAL SEGURO INMUEBLE : 0.39% 1.08% TASA ANUAL SEGURO INMUEBLE : 0.39% TASA ANUAL SEGURO DESGRAVAMEN TASA ANUAL SEGURO DESGRAVAMEN FECHA DESEMBOLSO: 21/10/23 FECHA DE EMISION CRONOGRAMA: 14/02/24 CUOTAS POR PAGAR: 27 DIAS DE GRACIA : 0 0.39% TASA ANUAL SEGURO DESGRAVAMEN : 1.08% TASA ANUAL SEGURO INMUEBLE : FECHA DESEMBOLSO: 21/10/23 FECHA DE EMISION CRONOGRAMA: 14/02/24 CUOTAS POR PAGAR: 27 DIAS DE GRACIA : 0 0.39% TASA ANUAL SEGURO DESGRAVAMEN : 1.08% TASA ANUAL SEGURO INMUEBLE : FECHA DESEMBOLSO: 21/10/23 FECHA DE EMISION CRONOGRAMA: 14/02/24 CUOTAS POR PAGAR: 27 DIAS DE GRACIA : 0 FECHA DESEMBOLSO: 21/10/23 FECHA DE EMISION CRONOGRAMA: 14/02/24 CUOTAS POR PAGAR: 27 DIAS DE GRACIA : 0 PERIODICIDAD : MENSUAL PERIODICIDAD : MENSUAL PERIODICIDAD : MENSUAL IMPORTE TOTAL A PAGAR (2) : 270,480.91 IMPORTE TOTAL INTERESES (3) : 26,714.57 PERIODICIDAD : MENSUAL IMPORTE TOTAL A PAGAR (2) : 270,480.91 IMPORTE TOTAL INTERESES (3) : 26,714.57 IMPORTE TOTAL A PAGAR (2) : 270,480.91 IMPORTE TOTAL INTERESES (3) : 26,714.57 IMPORTE TOTAL A PAGAR (2) : 270,480.91 IMPORTE TOTAL INTERESES (3) : 26,714.57 FUNCIONARIO NEG. : 28564 DESGRAVAMEN CREDITO NEGOCIOS (POLIZA 15319) FUNCIONARIO NEG. : 28564 DESGRAVAMEN CREDITO NEGOCIOS (POLIZA 15319) FUNCIONARIO NEG. : 28564 DESGRAVAMEN CREDITO NEGOCIOS (POLIZA 15319) FUNCIONARIO NEG. : 28564 DESGRAVAMEN CREDITO NEGOCIOS (POLIZA 15319) (1) LA TASA DE INTERES COMPENSATORIA EFECTIVA ANUAL (TEA) ES FIJA Y SE CALCULA TOMANDO EN CUENTA UN A O BASE DE 360 DIAS. (1) LA TASA DE INTERES COMPENSATORIA EFECTIVA ANUAL (TEA) ES FIJA Y SE CALCULA TOMANDO EN CUENTA UN A O BASE DE 360 DIAS. (2) IMPORTE TOTAL A PAGAR: INCLUYE LA SUMA DEL TOTAL DE CUOTAS MAS INTERESES FINANCIADOS Y SEGUROS FINANCIADOS. (2) IMPORTE TOTAL A PAGAR: INCLUYE LA SUMA DEL TOTAL DE CUOTAS MAS INTERESES FINANCIADOS Y SEGUROS FINANCIADOS. (3) IMPORTE TOTAL INTERESES: SUMA DEL TOTAL DE INTERESES COMPENSATORIOS ( CONSIDERA LOS INTERESES FINANCIADOS) . (3) IMPORTE TOTAL INTERESES: SUMA DEL TOTAL DE INTERESES COMPENSATORIOS ( CONSIDERA LOS INTERESES FINANCIADOS) . EL DETALLE DE PAGOS REALIZADOS PUEDE INCLUIR SEGURO DE DESGRAVAMEN O ADMINISTRACION DE POLIZA ENDOSADA. EL DETALLE DE PAGOS REALIZADOS PUEDE INCLUIR SEGURO DE DESGRAVAMEN O ADMINISTRACION DE POLIZA ENDOSADA. EL BANCO SE RESERVA EL DERECHO DE COBRAR PORTES, SEGUROS O INTERESES GENERADOS, NO COBRADOS HASTA LA EMISION DE ESTE CRONOGRAMA. EL BANCO SE RESERVA EL DERECHO DE COBRAR PORTES, SEGUROS O INTERESES GENERADOS, NO COBRADOS HASTA LA EMISION DE ESTE CRONOGRAMA. ESTE CRONOGRAMA SE ELABORA BAJO EL SUPUESTO DEL CUMPLIMIENTO DEL PAGO DE LAS CUOTAS EN LAS FECHAS INDICADAS. CUALQUIER ALTERACION EN LOS PAGOS O EN LAS CONDICIONES DEL CREDITO, DEJA SIN EFECTO ESTE DOCUMENTO. ESTE CRONOGRAMA SE ELABORA BAJO EL SUPUESTO DEL CUMPLIMIENTO DEL PAGO DE LAS CUOTAS EN LAS FECHAS INDICADAS. CUALQUIER ALTERACION EN LOS PAGOS O EN LAS CONDICIONES DEL CREDITO, DEJA SIN EFECTO ESTE DOCUMENTO. SI TUVIERA ALGUNA CONSULTA SIRVASE COMUNICARSE CON SU FUNCIONARIO DE NEGOCIOS O A BANCA POR TELEFONO VIABCP AL 3119898. SI TUVIERA ALGUNA CONSULTA SIRVASE COMUNICARSE CON SU FUNCIONARIO DE NEGOCIOS O A BANCA POR TELEFONO VIABCP AL 3119898.
                KEY VALUE PAIRS:
                {"CREDITO NRO": "100-285-0000-00000009400201", "PRODUCTO": "CREDITO NEGOCIOS\nSOLES", "IMPORTE DESEMBOLSADO": "240,950.00", "IMPORTE PRESTAMO": "240,950.00", "TASA DE INTERES COMPENSATORIA EFECTIVA ANUAL (1)": "12.70%", "TASA DE COSTO EFECTIVO ANUAL REMANENTE": "14.17%", "FECHA DESEMBOLSO": "21/10/23", "FECHA DE EMISION CRONOGRAMA": "14/02/24", "CUOTAS POR PAGAR": "27", "DIAS DE GRACIA": "0", "TASA ANUAL SEGURO DESGRAVAMEN": "1.08%", "TASA ANUAL SEGURO INMUEBLE": "0.39%", "PERIODICIDAD": "MENSUAL", "IMPORTE TOTAL A PAGAR (2)": "270,480.91", "IMPORTE TOTAL INTERESES (3)": "26,714.57", "FUNCIONARIO NEG.": "28564", "DESGRAVAMEN": "CREDITO NEGOCIOS (POLIZA 15319)"}
                TABLES:
                {"0": [{"row": 0, "column": 0, "content": "FECHA OPERAC."}, {"row": 0, "column": 1, "content": "SALDO CAPITAL"}, {"row": 0, "column": 2, "content": "AMORTIZACION"}, {"row": 0, "column": 3, "content": "INTERESES"}, {"row": 0, "column": 4, "content": "SEGURO DESGR."}, {"row": 0, "column": 5, "content": "SEGURO\nBIEN"}, {"row": 0, "column": 6, "content": "COMISIONES"}, {"row": 0, "column": 7, "content": "PAGADO"}, {"row": 1, "column": 0, "content": "211123"}, {"row": 1, "column": 1, "content": "240,950.00"}, {"row": 1, "column": 2, "content": "2,724.78"}, {"row": 1, "column": 3, "content": "2,573.49"}, {"row": 1, "column": 4, "content": "221.01"}, {"row": 1, "column": 5, "content": "14.95"}, {"row": 1, "column": 6, "content": "0.00"}, {"row": 1, "column": 7, "content": "5,534.23"}, {"row": 2, "column": 0, "content": "211123"}, {"row": 2, "column": 1, "content": "238,225.22"}, {"row": 2, "column": 2, "content": "50,000.00"}, {"row": 2, "column": 3, "content": "0.00"}, {"row": 2, "column": 4, "content": "0.00"}, {"row": 2, "column": 5, "content": "0.00"}, {"row": 2, "column": 6, "content": "0.00"}, {"row": 2, "column": 7, "content": "50,000.00"}, {"row": 3, "column": 0, "content": "211223"}, {"row": 3, "column": 1, "content": "188,225.22"}, {"row": 3, "column": 2, "content": "3,467.96"}, {"row": 3, "column": 3, "content": "1,884.71"}, {"row": 3, "column": 4, "content": "167.08"}, {"row": 3, "column": 5, "content": "14.48"}, {"row": 3, "column": 6, "content": "0.00"}, {"row": 3, "column": 7, "content": "5,534.23"}, {"row": 4, "column": 0, "content": "200124"}, {"row": 4, "column": 1, "content": "184,757.26"}, {"row": 4, "column": 2, "content": "3,438.14"}, {"row": 4, "column": 3, "content": "1,911.66"}, {"row": 4, "column": 4, "content": "169.47"}, {"row": 4, "column": 5, "content": "14.96"}, {"row": 4, "column": 6, "content": "0.00"}, {"row": 4, "column": 7, "content": "5,534.23"}, {"row": 5, "column": 0, "content": "080224"}, {"row": 5, "column": 1, "content": "181,319.12"}, {"row": 5, "column": 2, "content": "3,478.12"}, {"row": 5, "column": 3, "content": "1,874.93"}, {"row": 5, "column": 4, "content": "166.22"}, {"row": 5, "column": 5, "content": "14.96"}, {"row": 5, "column": 6, "content": "0.00"}, {"row": 5, "column": 7, "content": "5,534.23"}, {"row": 6, "column": 0, "content": "080224"}, {"row": 6, "column": 1, "content": "177,841.00"}, {"row": 6, "column": 2, "content": "50,500.00"}, {"row": 6, "column": 3, "content": "0.00"}, {"row": 6, "column": 4, "content": "0.00"}, {"row": 6, "column": 5, "content": "0.00"}, {"row": 6, "column": 6, "content": "0.00"}, {"row": 6, "column": 7, "content": "50,500.00"}, {"row": 7, "column": 0, "content": "TOTAL"}, {"row": 7, "column": 1, "content": "PAGADO"}, {"row": 7, "column": 2, "content": "113,609.00"}, {"row": 7, "column": 4, "content": "723.78"}, {"row": 7, "column": 6, "content": "0.00"}, {"row": 8, "column": 3, "content": "8,244.79"}, {"row": 8, "column": 5, "content": "59.35"}, {"row": 8, "column": 7, "content": "122,636.92"}], "1": [{"row": 0, "column": 0, "content": "PROXIMO VENCIM."}, {"row": 0, "column": 1, "content": "SALDO CAPITAL"}, {"row": 0, "column": 2, "content": "AMORTIZACION"}, {"row": 0, "column": 3, "content": "INTERESES"}, {"row": 0, "column": 4, "content": "SEGURO DESGR ."}, {"row": 0, "column": 5, "content": "SEGURO BIEN"}, {"row": 0, "column": 6, "content": "COMISIONES"}, {"row": 0, "column": 7, "content": "CUOTA"}, {"row": 1, "column": 0, "content": "210324"}, {"row": 1, "column": 1, "content": "127,341.00"}, {"row": 1, "column": 2, "content": "4,433.35"}, {"row": 1, "column": 3, "content": "998.37"}, {"row": 1, "column": 4, "content": "88.51"}, {"row": 1, "column": 5, "content": "14.00"}, {"row": 1, "column": 6, "content": "0.00"}, {"row": 1, "column": 7, "content": "5,534.23"}, {"row": 2, "column": 0, "content": "210424"}, {"row": 2, "column": 1, "content": "122,907.65"}, {"row": 2, "column": 2, "content": "4,134.82"}, {"row": 2, "column": 3, "content": "1,271.71"}, {"row": 2, "column": 4, "content": "112.74"}, {"row": 2, "column": 5, "content": "14.96"}, {"row": 2, "column": 6, "content": "0.00"}, {"row": 2, "column": 7, "content": "5,534.23"}, {"row": 3, "column": 0, "content": "210524"}, {"row": 3, "column": 1, "content": "118,772.83"}, {"row": 3, "column": 2, "content": "4,225.04"}, {"row": 3, "column": 3, "content": "1,189.28"}, {"row": 3, "column": 4, "content": "105.43"}, {"row": 3, "column": 5, "content": "14.48"}, {"row": 3, "column": 6, "content": "0.00"}, {"row": 3, "column": 7, "content": "5,534.23"}, {"row": 4, "column": 0, "content": "210624"}, {"row": 4, "column": 1, "content": "114,547.80"}, {"row": 4, "column": 2, "content": "4,228.99"}, {"row": 4, "column": 3, "content": "1,185.21"}, {"row": 4, "column": 4, "content": "105.07"}, {"row": 4, "column": 5, "content": "14.96"}, {"row": 4, "column": 6, "content": "0.00"}, {"row": 4, "column": 7, "content": "5,534.23"}, {"row": 5, "column": 0, "content": "210724"}, {"row": 5, "column": 1, "content": "110,318.81"}, {"row": 5, "column": 2, "content": "4,317.19"}, {"row": 5, "column": 3, "content": "1,104.63"}, {"row": 5, "column": 4, "content": "97.93"}, {"row": 5, "column": 5, "content": "14.48"}, {"row": 5, "column": 6, "content": "0.00"}, {"row": 5, "column": 7, "content": "5,534.23"}, {"row": 6, "column": 0, "content": "210824"}, {"row": 6, "column": 1, "content": "106,001.62"}, {"row": 6, "column": 2, "content": "4,325.26"}, {"row": 6, "column": 3, "content": "1,096.78"}, {"row": 6, "column": 4, "content": "97.23"}, {"row": 6, "column": 5, "content": "14.96"}, {"row": 6, "column": 6, "content": "0.00"}, {"row": 6, "column": 7, "content": "5,534.23"}, {"row": 7, "column": 0, "content": "210924"}, {"row": 7, "column": 1, "content": "101,676.37"}, {"row": 7, "column": 2, "content": "4,373.98"}, {"row": 7, "column": 3, "content": "1,052.03"}, {"row": 7, "column": 4, "content": "93.26"}, {"row": 7, "column": 5, "content": "14.96"}, {"row": 7, "column": 6, "content": "0.00"}, {"row": 7, "column": 7, "content": "5,534.23"}, {"row": 8, "column": 0, "content": "211024"}, {"row": 8, "column": 1, "content": "97,302.40"}, {"row": 8, "column": 2, "content": "4,459.08"}, {"row": 8, "column": 3, "content": "974.30"}, {"row": 8, "column": 4, "content": "86.37"}, {"row": 8, "column": 5, "content": "14.48"}, {"row": 8, "column": 6, "content": "0.00"}, {"row": 8, "column": 7, "content": "5,534.23"}, {"row": 9, "column": 0, "content": "211124"}, {"row": 9, "column": 1, "content": "92,843.32"}, {"row": 9, "column": 2, "content": "4,473.47"}, {"row": 9, "column": 3, "content": "960.64"}, {"row": 9, "column": 4, "content": "85.16"}, {"row": 9, "column": 5, "content": "14.96"}, {"row": 9, "column": 6, "content": "0.00"}, {"row": 9, "column": 7, "content": "5,534.23"}, {"row": 10, "column": 0, "content": "211224"}, {"row": 10, "column": 1, "content": "88,369.85"}, {"row": 10, "column": 2, "content": "4,556.45"}, {"row": 10, "column": 3, "content": "884.85"}, {"row": 10, "column": 4, "content": "78.44"}, {"row": 10, "column": 5, "content": "14.48"}, {"row": 10, "column": 6, "content": "0.00"}, {"row": 10, "column": 7, "content": "5,534.23"}, {"row": 11, "column": 0, "content": "210125"}, {"row": 11, "column": 1, "content": "83,813.40"}, {"row": 11, "column": 2, "content": "4,575.19"}, {"row": 11, "column": 3, "content": "867.20"}, {"row": 11, "column": 4, "content": "76.88"}, {"row": 11, "column": 5, "content": "14.96"}, {"row": 11, "column": 6, "content": "0.00"}, {"row": 11, "column": 7, "content": "5,534.23"}, {"row": 12, "column": 0, "content": "210225"}, {"row": 12, "column": 1, "content": "79,238.22"}, {"row": 12, "column": 2, "content": "4,626.72"}, {"row": 12, "column": 3, "content": "819.87"}, {"row": 12, "column": 4, "content": "72.68"}, {"row": 12, "column": 5, "content": "14.96"}, {"row": 12, "column": 6, "content": "0.00"}, {"row": 12, "column": 7, "content": "5,534.23"}, {"row": 13, "column": 0, "content": "210325"}, {"row": 13, "column": 1, "content": "74,611.50"}, {"row": 13, "column": 2, "content": "4,761.62"}, {"row": 13, "column": 3, "content": "697.29"}, {"row": 13, "column": 4, "content": "61.82"}, {"row": 13, "column": 5, "content": "13.51"}, {"row": 13, "column": 6, "content": "0.00"}, {"row": 13, "column": 7, "content": "5,534.23"}, {"row": 14, "column": 0, "content": "210425"}, {"row": 14, "column": 1, "content": "69,849.89"}, {"row": 14, "column": 2, "content": "4,732.47"}, {"row": 14, "column": 3, "content": "722.73"}, {"row": 14, "column": 4, "content": "64.07"}, {"row": 14, "column": 5, "content": "14.96"}, {"row": 14, "column": 6, "content": "0.00"}, {"row": 14, "column": 7, "content": "5,534.23"}], "2": [{"row": 0, "column": 0, "content": "FECHA OPERAC."}, {"row": 0, "column": 1, "content": "SALDO CAPITAL"}, {"row": 0, "column": 2, "content": "AMORTIZACION"}, {"row": 0, "column": 3, "content": "INTERESES"}, {"row": 0, "column": 4, "content": "SEGURO DESGR."}, {"row": 0, "column": 5, "content": "SEGURO\nBIEN"}, {"row": 0, "column": 6, "content": "COMISIONES"}, {"row": 0, "column": 7, "content": "PAGADO"}, {"row": 1, "column": 0, "content": "211123"}, {"row": 1, "column": 1, "content": "240,950.00"}, {"row": 1, "column": 2, "content": "2,724.78"}, {"row": 1, "column": 3, "content": "2,573.49"}, {"row": 1, "column": 4, "content": "221.01"}, {"row": 1, "column": 5, "content": "14.95"}, {"row": 1, "column": 6, "content": "0.00"}, {"row": 1, "column": 7, "content": "5,534.23"}, {"row": 2, "column": 0, "content": "211123"}, {"row": 2, "column": 1, "content": "238,225.22"}, {"row": 2, "column": 2, "content": "50,000.00"}, {"row": 2, "column": 3, "content": "0.00"}, {"row": 2, "column": 4, "content": "0.00"}, {"row": 2, "column": 5, "content": "0.00"}, {"row": 2, "column": 6, "content": "0.00"}, {"row": 2, "column": 7, "content": "50,000.00"}, {"row": 3, "column": 0, "content": "211223"}, {"row": 3, "column": 1, "content": "188,225.22"}, {"row": 3, "column": 2, "content": "3,467.96"}, {"row": 3, "column": 3, "content": "1,884.71"}, {"row": 3, "column": 4, "content": "167.08"}, {"row": 3, "column": 5, "content": "14.48"}, {"row": 3, "column": 6, "content": "0.00"}, {"row": 3, "column": 7, "content": "5,534.23"}, {"row": 4, "column": 0, "content": "200124"}, {"row": 4, "column": 1, "content": "184,757.26"}, {"row": 4, "column": 2, "content": "3,438.14"}, {"row": 4, "column": 3, "content": "1,911.66"}, {"row": 4, "column": 4, "content": "169.47"}, {"row": 4, "column": 5, "content": "14.96"}, {"row": 4, "column": 6, "content": "0.00"}, {"row": 4, "column": 7, "content": "5,534.23"}, {"row": 5, "column": 0, "content": "080224"}, {"row": 5, "column": 1, "content": "181,319.12"}, {"row": 5, "column": 2, "content": "3,478.12"}, {"row": 5, "column": 3, "content": "1,874.93"}, {"row": 5, "column": 4, "content": "166.22"}, {"row": 5, "column": 5, "content": "14.96"}, {"row": 5, "column": 6, "content": "0.00"}, {"row": 5, "column": 7, "content": "5,534.23"}, {"row": 6, "column": 0, "content": "080224"}, {"row": 6, "column": 1, "content": "177,841.00"}, {"row": 6, "column": 2, "content": "50,500.00"}, {"row": 6, "column": 3, "content": "0.00"}, {"row": 6, "column": 4, "content": "0.00"}, {"row": 6, "column": 5, "content": "0.00"}, {"row": 6, "column": 6, "content": "0.00"}, {"row": 6, "column": 7, "content": "50,500.00"}, {"row": 7, "column": 0, "content": "TOTAL"}, {"row": 7, "column": 1, "content": "PAGADO"}, {"row": 7, "column": 2, "content": "113,609.00"}, {"row": 7, "column": 4, "content": "723.78"}, {"row": 7, "column": 6, "content": "0.00"}, {"row": 8, "column": 3, "content": "8,244.79"}, {"row": 8, "column": 5, "content": "59.35"}, {"row": 8, "column": 7, "content": "122,636.92"}], "3": [{"row": 0, "column": 0, "content": "PROXIMO VENCIM."}, {"row": 0, "column": 1, "content": "SALDO CAPITAL"}, {"row": 0, "column": 2, "content": "AMORTIZACION"}, {"row": 0, "column": 3, "content": "INTERESES"}, {"row": 0, "column": 4, "content": "SEGURO DESGR ."}, {"row": 0, "column": 5, "content": "SEGURO BIEN"}, {"row": 0, "column": 6, "content": "COMISIONES"}, {"row": 0, "column": 7, "content": "CUOTA"}, {"row": 1, "column": 0, "content": "210324"}, {"row": 1, "column": 1, "content": "127,341.00"}, {"row": 1, "column": 2, "content": "4,433.35"}, {"row": 1, "column": 3, "content": "998.37"}, {"row": 1, "column": 4, "content": "88.51"}, {"row": 1, "column": 5, "content": "14.00"}, {"row": 1, "column": 6, "content": "0.00"}, {"row": 1, "column": 7, "content": "5,534.23"}, {"row": 2, "column": 0, "content": "210424"}, {"row": 2, "column": 1, "content": "122,907.65"}, {"row": 2, "column": 2, "content": "4,134.82"}, {"row": 2, "column": 3, "content": "1,271.71"}, {"row": 2, "column": 4, "content": "112.74"}, {"row": 2, "column": 5, "content": "14.96"}, {"row": 2, "column": 6, "content": "0.00"}, {"row": 2, "column": 7, "content": "5,534.23"}, {"row": 3, "column": 0, "content": "210524"}, {"row": 3, "column": 1, "content": "118,772.83"}, {"row": 3, "column": 2, "content": "4,225.04"}, {"row": 3, "column": 3, "content": "1,189.28"}, {"row": 3, "column": 4, "content": "105.43"}, {"row": 3, "column": 5, "content": "14.48"}, {"row": 3, "column": 6, "content": "0.00"}, {"row": 3, "column": 7, "content": "5,534.23"}, {"row": 4, "column": 0, "content": "210624"}, {"row": 4, "column": 1, "content": "114,547.80"}, {"row": 4, "column": 2, "content": "4,228.99"}, {"row": 4, "column": 3, "content": "1,185.21"}, {"row": 4, "column": 4, "content": "105.07"}, {"row": 4, "column": 5, "content": "14.96"}, {"row": 4, "column": 6, "content": "0.00"}, {"row": 4, "column": 7, "content": "5,534.23"}, {"row": 5, "column": 0, "content": "210724"}, {"row": 5, "column": 1, "content": "110,318.81"}, {"row": 5, "column": 2, "content": "4,317.19"}, {"row": 5, "column": 3, "content": "1,104.63"}, {"row": 5, "column": 4, "content": "97.93"}, {"row": 5, "column": 5, "content": "14.48"}, {"row": 5, "column": 6, "content": "0.00"}, {"row": 5, "column": 7, "content": "5,534.23"}, {"row": 6, "column": 0, "content": "210824"}, {"row": 6, "column": 1, "content": "106,001.62"}, {"row": 6, "column": 2, "content": "4,325.26"}, {"row": 6, "column": 3, "content": "1,096.78"}, {"row": 6, "column": 4, "content": "97.23"}, {"row": 6, "column": 5, "content": "14.96"}, {"row": 6, "column": 6, "content": "0.00"}, {"row": 6, "column": 7, "content": "5,534.23"}, {"row": 7, "column": 0, "content": "210924"}, {"row": 7, "column": 1, "content": "101,676.37"}, {"row": 7, "column": 2, "content": "4,373.98"}, {"row": 7, "column": 3, "content": "1,052.03"}, {"row": 7, "column": 4, "content": "93.26"}, {"row": 7, "column": 5, "content": "14.96"}, {"row": 7, "column": 6, "content": "0.00"}, {"row": 7, "column": 7, "content": "5,534.23"}, {"row": 8, "column": 0, "content": "211024"}, {"row": 8, "column": 1, "content": "97,302.40"}, {"row": 8, "column": 2, "content": "4,459.08"}, {"row": 8, "column": 3, "content": "974.30"}, {"row": 8, "column": 4, "content": "86.37"}, {"row": 8, "column": 5, "content": "14.48"}, {"row": 8, "column": 6, "content": "0.00"}, {"row": 8, "column": 7, "content": "5,534.23"}, {"row": 9, "column": 0, "content": "211124"}, {"row": 9, "column": 1, "content": "92,843.32"}, {"row": 9, "column": 2, "content": "4,473.47"}, {"row": 9, "column": 3, "content": "960.64"}, {"row": 9, "column": 4, "content": "85.16"}, {"row": 9, "column": 5, "content": "14.96"}, {"row": 9, "column": 6, "content": "0.00"}, {"row": 9, "column": 7, "content": "5,534.23"}, {"row": 10, "column": 0, "content": "211224"}, {"row": 10, "column": 1, "content": "88,369.85"}, {"row": 10, "column": 2, "content": "4,556.45"}, {"row": 10, "column": 3, "content": "884.85"}, {"row": 10, "column": 4, "content": "78.44"}, {"row": 10, "column": 5, "content": "14.48"}, {"row": 10, "column": 6, "content": "0.00"}, {"row": 10, "column": 7, "content": "5,534.23"}, {"row": 11, "column": 0, "content": "210125"}, {"row": 11, "column": 1, "content": "83,813.40"}, {"row": 11, "column": 2, "content": "4,575.19"}, {"row": 11, "column": 3, "content": "867.20"}, {"row": 11, "column": 4, "content": "76.88"}, {"row": 11, "column": 5, "content": "14.96"}, {"row": 11, "column": 6, "content": "0.00"}, {"row": 11, "column": 7, "content": "5,534.23"}, {"row": 12, "column": 0, "content": "210225"}, {"row": 12, "column": 1, "content": "79,238.22"}, {"row": 12, "column": 2, "content": "4,626.72"}, {"row": 12, "column": 3, "content": "819.87"}, {"row": 12, "column": 4, "content": "72.68"}, {"row": 12, "column": 5, "content": "14.96"}, {"row": 12, "column": 6, "content": "0.00"}, {"row": 12, "column": 7, "content": "5,534.23"}, {"row": 13, "column": 0, "content": "210325"}, {"row": 13, "column": 1, "content": "74,611.50"}, {"row": 13, "column": 2, "content": "4,761.62"}, {"row": 13, "column": 3, "content": "697.29"}, {"row": 13, "column": 4, "content": "61.82"}, {"row": 13, "column": 5, "content": "13.51"}, {"row": 13, "column": 6, "content": "0.00"}, {"row": 13, "column": 7, "content": "5,534.23"}, {"row": 14, "column": 0, "content": "210425"}, {"row": 14, "column": 1, "content": "69,849.89"}, {"row": 14, "column": 2, "content": "4,732.47"}, {"row": 14, "column": 3, "content": "722.73"}, {"row": 14, "column": 4, "content": "64.07"}, {"row": 14, "column": 5, "content": "14.96"}, {"row": 14, "column": 6, "content": "0.00"}, {"row": 14, "column": 7, "content": "5,534.23"}], "4": [{"row": 0, "column": 0, "content": "PROXIMO VENCIM."}, {"row": 0, "column": 1, "content": "SALDO CAPITAL"}, {"row": 0, "column": 2, "content": "AMORTIZACI\u00d3N"}, {"row": 0, "column": 3, "content": "INTERESES"}, {"row": 0, "column": 4, "content": "SEGURO DESGR."}, {"row": 0, "column": 5, "content": "SEGURO BIEN"}, {"row": 0, "column": 6, "content": "COMISIONES"}, {"row": 0, "column": 7, "content": "CUOTA"}, {"row": 1, "column": 0, "content": "210525"}, {"row": 1, "column": 1, "content": "65,117.42"}, {"row": 1, "column": 2, "content": "4,809.92"}, {"row": 1, "column": 3, "content": "652.03"}, {"row": 1, "column": 4, "content": "57.80"}, {"row": 1, "column": 5, "content": "14.48"}, {"row": 1, "column": 6, "content": "0.00"}, {"row": 1, "column": 7, "content": "5,534.23"}, {"row": 2, "column": 0, "content": "210625"}, {"row": 2, "column": 1, "content": "60,307.50"}, {"row": 2, "column": 2, "content": "4,839.96"}, {"row": 2, "column": 3, "content": "623.99"}, {"row": 2, "column": 4, "content": "55.32"}, {"row": 2, "column": 5, "content": "14.96"}, {"row": 2, "column": 6, "content": "0.00"}, {"row": 2, "column": 7, "content": "5,534.23"}, {"row": 3, "column": 0, "content": "210725"}, {"row": 3, "column": 1, "content": "55,467.55"}, {"row": 3, "column": 2, "content": "4,915.11"}, {"row": 3, "column": 3, "content": "555.40"}, {"row": 3, "column": 4, "content": "49.24"}, {"row": 3, "column": 5, "content": "14.48"}, {"row": 3, "column": 6, "content": "0.00"}, {"row": 3, "column": 7, "content": "5,534.23"}, {"row": 4, "column": 0, "content": "210825"}, {"row": 4, "column": 1, "content": "50,552.44"}, {"row": 4, "column": 2, "content": "4,949.84"}, {"row": 4, "column": 3, "content": "523.06"}, {"row": 4, "column": 4, "content": "46.37"}, {"row": 4, "column": 5, "content": "14.96"}, {"row": 4, "column": 6, "content": "0.00"}, {"row": 4, "column": 7, "content": "5,534.23"}, {"row": 5, "column": 0, "content": "210925"}, {"row": 5, "column": 1, "content": "45,602.60"}, {"row": 5, "column": 2, "content": "5,005.60"}, {"row": 5, "column": 3, "content": "471.84"}, {"row": 5, "column": 4, "content": "41.83"}, {"row": 5, "column": 5, "content": "14.96"}, {"row": 5, "column": 6, "content": "0.00"}, {"row": 5, "column": 7, "content": "5,534.23"}, {"row": 6, "column": 0, "content": "211025"}, {"row": 6, "column": 1, "content": "40,597.01"}, {"row": 6, "column": 2, "content": "5,077.21"}, {"row": 6, "column": 3, "content": "406.50"}, {"row": 6, "column": 4, "content": "36.04"}, {"row": 6, "column": 5, "content": "14.48"}, {"row": 6, "column": 6, "content": "0.00"}, {"row": 6, "column": 7, "content": "5,534.23"}, {"row": 7, "column": 0, "content": "211125"}, {"row": 7, "column": 1, "content": "35,519.80"}, {"row": 7, "column": 2, "content": "5,119.17"}, {"row": 7, "column": 3, "content": "367.52"}, {"row": 7, "column": 4, "content": "32.58"}, {"row": 7, "column": 5, "content": "14.96"}, {"row": 7, "column": 6, "content": "0.00"}, {"row": 7, "column": 7, "content": "5,534.23"}, {"row": 8, "column": 0, "content": "211225"}, {"row": 8, "column": 1, "content": "30,400.63"}, {"row": 8, "column": 2, "content": "5,188.36"}, {"row": 8, "column": 3, "content": "304.40"}, {"row": 8, "column": 4, "content": "26.99"}, {"row": 8, "column": 5, "content": "14.48"}, {"row": 8, "column": 6, "content": "0.00"}, {"row": 8, "column": 7, "content": "5,534.23"}, {"row": 9, "column": 0, "content": "210126"}, {"row": 9, "column": 1, "content": "25,212.27"}, {"row": 9, "column": 2, "content": "5,235.28"}, {"row": 9, "column": 3, "content": "260.87"}, {"row": 9, "column": 4, "content": "23.13"}, {"row": 9, "column": 5, "content": "14.96"}, {"row": 9, "column": 6, "content": "0.00"}, {"row": 9, "column": 7, "content": "5,534.23"}, {"row": 10, "column": 0, "content": "210226"}, {"row": 10, "column": 1, "content": "19,977.00"}, {"row": 10, "column": 2, "content": "5,294.25"}, {"row": 10, "column": 3, "content": "206.70"}, {"row": 10, "column": 4, "content": "18.32"}, {"row": 10, "column": 5, "content": "14.96"}, {"row": 10, "column": 6, "content": "0.00"}, {"row": 10, "column": 7, "content": "5,534.23"}, {"row": 11, "column": 0, "content": "210326"}, {"row": 11, "column": 1, "content": "14,682.76"}, {"row": 11, "column": 2, "content": "5,371.33"}, {"row": 11, "column": 3, "content": "137.22"}, {"row": 11, "column": 4, "content": "12.16"}, {"row": 11, "column": 5, "content": "13.51"}, {"row": 11, "column": 6, "content": "0.00"}, {"row": 11, "column": 7, "content": "5,534.23"}, {"row": 12, "column": 0, "content": "210426"}, {"row": 12, "column": 1, "content": "9,311.43"}, {"row": 12, "column": 2, "content": "5,414.39"}, {"row": 12, "column": 3, "content": "96.34"}, {"row": 12, "column": 4, "content": "8.54"}, {"row": 12, "column": 5, "content": "14.96"}, {"row": 12, "column": 6, "content": "0.00"}, {"row": 12, "column": 7, "content": "5,534.23"}, {"row": 13, "column": 0, "content": "210526"}, {"row": 13, "column": 1, "content": "3,897.05"}, {"row": 13, "column": 2, "content": "3,897.05"}, {"row": 13, "column": 3, "content": "39.02"}, {"row": 13, "column": 4, "content": "3.46"}, {"row": 13, "column": 5, "content": "14.48"}, {"row": 13, "column": 6, "content": "0.00"}, {"row": 13, "column": 7, "content": "3,954.01"}, {"row": 14, "column": 0, "content": "TOTAL POR PAGAR"}, {"row": 14, "column": 2, "content": "127,341.10"}, {"row": 14, "column": 4, "content": "1,637.37"}, {"row": 14, "column": 6, "content": "0.00"}, {"row": 15, "column": 3, "content": "18,469.78"}, {"row": 15, "column": 5, "content": "395.74"}, {"row": 15, "column": 7, "content": "147,843.99"}], "5": [{"row": 0, "column": 0, "content": "PROXIMO VENCIM."}, {"row": 0, "column": 1, "content": "SALDO CAPITAL"}, {"row": 0, "column": 2, "content": "AMORTIZACI\u00d3N"}, {"row": 0, "column": 3, "content": "INTERESES"}, {"row": 0, "column": 4, "content": "SEGURO DESGR."}, {"row": 0, "column": 5, "content": "SEGURO BIEN"}, {"row": 0, "column": 6, "content": "COMISIONES"}, {"row": 0, "column": 7, "content": "CUOTA"}, {"row": 1, "column": 0, "content": "210525"}, {"row": 1, "column": 1, "content": "65,117.42"}, {"row": 1, "column": 2, "content": "4,809.92"}, {"row": 1, "column": 3, "content": "652.03"}, {"row": 1, "column": 4, "content": "57.80"}, {"row": 1, "column": 5, "content": "14.48"}, {"row": 1, "column": 6, "content": "0.00"}, {"row": 1, "column": 7, "content": "5,534.23"}, {"row": 2, "column": 0, "content": "210625"}, {"row": 2, "column": 1, "content": "60,307.50"}, {"row": 2, "column": 2, "content": "4,839.96"}, {"row": 2, "column": 3, "content": "623.99"}, {"row": 2, "column": 4, "content": "55.32"}, {"row": 2, "column": 5, "content": "14.96"}, {"row": 2, "column": 6, "content": "0.00"}, {"row": 2, "column": 7, "content": "5,534.23"}, {"row": 3, "column": 0, "content": "210725"}, {"row": 3, "column": 1, "content": "55,467.55"}, {"row": 3, "column": 2, "content": "4,915.11"}, {"row": 3, "column": 3, "content": "555.40"}, {"row": 3, "column": 4, "content": "49.24"}, {"row": 3, "column": 5, "content": "14.48"}, {"row": 3, "column": 6, "content": "0.00"}, {"row": 3, "column": 7, "content": "5,534.23"}, {"row": 4, "column": 0, "content": "210825"}, {"row": 4, "column": 1, "content": "50,552.44"}, {"row": 4, "column": 2, "content": "4,949.84"}, {"row": 4, "column": 3, "content": "523.06"}, {"row": 4, "column": 4, "content": "46.37"}, {"row": 4, "column": 5, "content": "14.96"}, {"row": 4, "column": 6, "content": "0.00"}, {"row": 4, "column": 7, "content": "5,534.23"}, {"row": 5, "column": 0, "content": "210925"}, {"row": 5, "column": 1, "content": "45,602.60"}, {"row": 5, "column": 2, "content": "5,005.60"}, {"row": 5, "column": 3, "content": "471.84"}, {"row": 5, "column": 4, "content": "41.83"}, {"row": 5, "column": 5, "content": "14.96"}, {"row": 5, "column": 6, "content": "0.00"}, {"row": 5, "column": 7, "content": "5,534.23"}, {"row": 6, "column": 0, "content": "211025"}, {"row": 6, "column": 1, "content": "40,597.01"}, {"row": 6, "column": 2, "content": "5,077.21"}, {"row": 6, "column": 3, "content": "406.50"}, {"row": 6, "column": 4, "content": "36.04"}, {"row": 6, "column": 5, "content": "14.48"}, {"row": 6, "column": 6, "content": "0.00"}, {"row": 6, "column": 7, "content": "5,534.23"}, {"row": 7, "column": 0, "content": "211125"}, {"row": 7, "column": 1, "content": "35,519.80"}, {"row": 7, "column": 2, "content": "5,119.17"}, {"row": 7, "column": 3, "content": "367.52"}, {"row": 7, "column": 4, "content": "32.58"}, {"row": 7, "column": 5, "content": "14.96"}, {"row": 7, "column": 6, "content": "0.00"}, {"row": 7, "column": 7, "content": "5,534.23"}, {"row": 8, "column": 0, "content": "211225"}, {"row": 8, "column": 1, "content": "30,400.63"}, {"row": 8, "column": 2, "content": "5,188.36"}, {"row": 8, "column": 3, "content": "304.40"}, {"row": 8, "column": 4, "content": "26.99"}, {"row": 8, "column": 5, "content": "14.48"}, {"row": 8, "column": 6, "content": "0.00"}, {"row": 8, "column": 7, "content": "5,534.23"}, {"row": 9, "column": 0, "content": "210126"}, {"row": 9, "column": 1, "content": "25,212.27"}, {"row": 9, "column": 2, "content": "5,235.28"}, {"row": 9, "column": 3, "content": "260.87"}, {"row": 9, "column": 4, "content": "23.13"}, {"row": 9, "column": 5, "content": "14.96"}, {"row": 9, "column": 6, "content": "0.00"}, {"row": 9, "column": 7, "content": "5,534.23"}, {"row": 10, "column": 0, "content": "210226"}, {"row": 10, "column": 1, "content": "19,977.00"}, {"row": 10, "column": 2, "content": "5,294.25"}, {"row": 10, "column": 3, "content": "206.70"}, {"row": 10, "column": 4, "content": "18.32"}, {"row": 10, "column": 5, "content": "14.96"}, {"row": 10, "column": 6, "content": "0.00"}, {"row": 10, "column": 7, "content": "5,534.23"}, {"row": 11, "column": 0, "content": "210326"}, {"row": 11, "column": 1, "content": "14,682.76"}, {"row": 11, "column": 2, "content": "5,371.33"}, {"row": 11, "column": 3, "content": "137.22"}, {"row": 11, "column": 4, "content": "12.16"}, {"row": 11, "column": 5, "content": "13.51"}, {"row": 11, "column": 6, "content": "0.00"}, {"row": 11, "column": 7, "content": "5,534.23"}, {"row": 12, "column": 0, "content": "210426"}, {"row": 12, "column": 1, "content": "9,311.43"}, {"row": 12, "column": 2, "content": "5,414.39"}, {"row": 12, "column": 3, "content": "96.34"}, {"row": 12, "column": 4, "content": "8.54"}, {"row": 12, "column": 5, "content": "14.96"}, {"row": 12, "column": 6, "content": "0.00"}, {"row": 12, "column": 7, "content": "5,534.23"}, {"row": 13, "column": 0, "content": "210526"}, {"row": 13, "column": 1, "content": "3,897.05"}, {"row": 13, "column": 2, "content": "3,897.05"}, {"row": 13, "column": 3, "content": "39.02"}, {"row": 13, "column": 4, "content": "3.46"}, {"row": 13, "column": 5, "content": "14.48"}, {"row": 13, "column": 6, "content": "0.00"}, {"row": 13, "column": 7, "content": "3,954.01"}, {"row": 14, "column": 0, "content": "TOTAL POR PAGAR"}, {"row": 14, "column": 2, "content": "127,341.10"}, {"row": 14, "column": 4, "content": "1,637.37"}, {"row": 14, "column": 6, "content": "0.00"}, {"row": 15, "column": 3, "content": "18,469.78"}, {"row": 15, "column": 5, "content": "395.74"}, {"row": 15, "column": 7, "content": "147,843.99"}]}
            
"""

In [ ]:
chain.invoke({"ocr_content": ocr_content})

In [22]:
import fitz  # PyMuPDF
import base64
import os

def transform_to_base64_image(file_path: str) -> str:
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() in ['.pdf']:
        # Convertir PDF a imagen
        pdf_document = fitz.open(file_path)
        page = pdf_document.load_page(0)
        pix = page.get_pixmap()
        image_bytes = pix.tobytes("jpg")
        image_data = base64.b64encode(image_bytes).decode("utf-8")
    elif file_extension.lower() in ['.jpg', '.jpeg', '.png']:
        # Leer y codificar la imagen a base64
        with open(file_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")
    else:
        raise ValueError("Unsupported file format")
    return image_data


In [9]:
template = """
## Task Goal
The task goal is to identify specific financial details from the image of a payroll slip.

## Task Instructions
You will be given an image of a payroll slip in Spanish. The image contains plain text, key-value pairs, and tables. Please identify and extract the following elements:

1. **Company**: Identify the employer or entity that issued the payroll slip.
2. **Net Salary**: Find the net salary that the employee receives. It may appear as "sueldo neto", "salario neto", "remuneración neta", "saldo neto", etc.
3. **Variable Amounts**: Identify all variable amounts listed in the payroll slip, such as "bonos" (bonuses), "renta de quinta categoría" (fifth category income), "utilidades" (profits), "vacaciones" (vacation), "seguro" (insurance), "aporte" (contribution), "comisión" (commission), "gratificación" (gratuity), "alimentos" (food), "asignación familiar" (family allowance), "Essalud" (social health insurance), "EPS" (private health insurance), "incentivo" (incentive), "retención" (retention), etc. Return this as a dictionary, indicating the concept and the amount.

If you cannot find a specific value, return an empty string ("").

## Task Output:
Provide the extracted information as follows:
- Company: [Company Name]
- Net Salary: [Net Salary Amount]
- Variable Amounts: [
                      "concept": "amount",
                      ...
                    ]

## Task Input:
"""

In [1]:
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

class Payment_Information(BaseModel):
    """Relevant information extracted from payment schedules"""

    bank: str = Field(description="The Bank that issued the loan")
    monthly_payment: str = Field(description="The monthly payment amount the user pays for their loan.")
    total_loan_amount: str = Field(description="The total amount loaned by the bank.")
    interest_rate: str = Field(description="Find the interest rate at which the loan was issued.")

class Salary_Information(BaseModel):
    """Relevant information extracted from salary slips"""

    company: str = Field(description="The employer or entity that issued the payroll slip.")
    net_salary: str = Field(description="The net salary that the employee receives.")
    variable_amounts: str = Field(description="All variable amounts listed in the payroll slip.")

payment_functions = [convert_pydantic_to_openai_function(Payment_Information)]
salary_functions = [convert_pydantic_to_openai_function(Salary_Information)]

c:\Users\julio\OneDrive\Documentos\Salto\project\read_docs_api\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [7]:
import base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
import httpx
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

file_path = r'C:\Users\julio\OneDrive\Documentos\Salto\project\metrics\Input\BOLETAS DE PAGO\Andreita\57C4263A-7602-4328-B9E0-C7E58D9E1CA8.pdf'

# Transformar archivo a imagen codificada en base64
image_data = encode_image(file_path)

prompt = ChatPromptTemplate.from_messages(
                                            [
                                                ("system", f"{template}"),
                                                (
                                                    "user",
                                                    [{"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data}"}],
                                                ),
                                            ]
                                        )
model = AzureChatOpenAI(
    openai_api_type='azure',
    openai_api_version="2024-02-15-preview",
    azure_deployment="oaisaltoibk4o",
    azure_endpoint="https://saltoeusibkoai01dev.openai.azure.com/",
    openai_api_key="d0a66ddc2ce24a0b96d1ab206233f079",
)
# Build chain
parser = JsonOutputFunctionsParser()
chain = prompt | model.bind(functions=salary_functions) | parser
# Invoke Azure OpenAI
result = chain.invoke({"image_data": image_data})

BadRequestError: Error code: 400 - {'error': {'code': 'BadRequest', 'message': 'Invalid image data.', 'param': None, 'type': None}}

In [13]:
print(chain)

first=ChatPromptTemplate(input_variables=['image_data'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n## Task Goal\nThe task goal is to identify specific financial details from the image of a payroll slip.\n\n## Task Instructions\nYou will be given an image of a payroll slip in Spanish. The image contains plain text, key-value pairs, and tables. Please identify and extract the following elements:\n\n1. **Company**: Identify the employer or entity that issued the payroll slip.\n2. **Net Salary**: Find the net salary that the employee receives. It may appear as "sueldo neto", "salario neto", "remuneración neta", "saldo neto", etc.\n3. **Variable Amounts**: Identify all variable amounts listed in the payroll slip, such as "bonos" (bonuses), "renta de quinta categoría" (fifth category income), "utilidades" (profits), "vacaciones" (vacation), "seguro" (insurance), "aporte" (contribution), "comisión" (commission), "gratificación" (gratuity), "al

In [14]:
a = str('hola')